In [49]:
%load_ext autoreload
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from keras.preprocessing import sequence
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
X_train = pickle.load( open( "data/X_train.pickle", "rb" ) )
y_train = np.array(pickle.load( open( "data/y_train.pickle", "rb" ) ))

In [5]:
X_train =sequence.pad_sequences(X_train, maxlen=16, dtype='float')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X_train, y_train, test_size=0.20, random_state=42)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [8]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=512)

Epoch 1/10
94355/94355 [==============================] - 12s - loss: 0.7956    
Epoch 2/10
94355/94355 [==============================] - 11s - loss: 0.5919    
Epoch 3/10
94355/94355 [==============================] - 11s - loss: 0.5648    
Epoch 4/10
94355/94355 [==============================] - 11s - loss: 0.5531    
Epoch 5/10
94355/94355 [==============================] - 12s - loss: 0.5447    
Epoch 6/10
94355/94355 [==============================] - 11s - loss: 0.5396    
Epoch 7/10
94355/94355 [==============================] - 12s - loss: 0.5357    
Epoch 8/10
94355/94355 [==============================] - 12s - loss: 0.5321    
Epoch 9/10
94355/94355 [==============================] - 12s - loss: 0.5296    
Epoch 10/10
94355/94355 [==============================] - 12s - loss: 0.5270    


In [15]:
preds = np.round(model.predict_proba(X_test), 2)

23584/23589 [============================>.] - ETA: 0s

In [80]:
score = calculate_score(y_test, preds, X_test)
print(score)

0.728955781784


In [81]:
fp = "models/lstm_{0}_SCORE_{1}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), np.round(score,5)) 
model.save(fp)